In [4]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_433021/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [6]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [7]:
analize = Analizer(0.4)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_9_17,0.435032,-1.294153,0.091682,-0.014956,0.067245,0.068783,1.361908,0.135772,0.117333,0.126552,0.101178,0.262265,-0.008872,0.264372,115.353597,271.349102,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
1,model_1_9_18,0.432656,-0.803729,0.031654,-0.090235,0.002106,0.069072,1.070771,0.144744,0.126036,0.135390,0.101481,0.262816,-0.013114,0.264927,115.345206,271.340711,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
2,model_1_9_16,0.406317,-1.282838,0.088937,-0.005034,0.069961,0.072279,1.355191,0.136182,0.116186,0.126184,0.104614,0.268847,-0.060147,0.271007,115.254447,271.249952,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
3,model_1_9_20,0.401550,-0.777637,0.025998,-0.444856,-0.152088,0.072859,1.055282,0.145590,0.167032,0.156311,0.100874,0.269925,-0.068660,0.272093,115.238452,271.233956,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
4,model_1_9_15,0.398697,-1.159466,0.178987,0.006474,0.124469,0.073207,1.281952,0.122722,0.114856,0.118788,0.105170,0.270567,-0.073756,0.272741,115.228938,271.224443,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
5,model_1_9_19,0.395644,-0.754619,0.046709,-0.448029,-0.142030,0.073578,1.041618,0.142494,0.167398,0.154946,0.101963,0.271253,-0.079208,0.273432,115.218809,271.214314,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
6,model_1_9_14,0.369788,-1.047474,0.223432,-0.020816,0.137325,0.076726,1.215469,0.116078,0.118011,0.117044,0.108819,0.276995,-0.125379,0.279220,115.135023,271.130528,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
7,model_1_9_21,0.338890,-1.011812,-0.083887,-0.895915,-0.404779,0.080488,1.194298,0.162015,0.219176,0.190595,0.102396,0.283704,-0.180553,0.285983,115.039297,271.034801,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
8,model_1_9_13,0.320360,-1.288811,0.170390,-0.027346,0.105324,0.082744,1.358737,0.124007,0.118766,0.121386,0.114389,0.287652,-0.213642,0.289963,114.984011,270.979516,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
9,model_1_9_12,0.318959,-1.259353,0.172456,-0.022140,0.108678,0.082914,1.341249,0.123698,0.118164,0.120931,0.114364,0.287949,-0.216145,0.290262,114.979891,270.975396,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
